In [3]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
# import sklearn

In [4]:
# read data
date_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%y')
medication_df = pd.read_csv('./medication.csv', parse_dates=['date'], date_parser=date_parser)
user_df = pd.read_csv('./users.csv', parse_dates=True)
blood_pressure_df = pd.read_csv('./blood_pressure[1].csv', parse_dates=['date'], date_parser=date_parser)

In [5]:
display(medication_df.head());  display(user_df.head()); display(blood_pressure_df.head())

,user_id,date,pill_detect
0,100000,2017-04-24,False
1,100000,2017-04-25,True
2,100000,2017-04-26,True
3,100000,2017-04-27,False
4,100000,2017-04-28,False


,user_id,gender,bmi
0,100000,female,39
1,100001,male,25
2,100002,female,28
3,100003,male,25
4,100004,female,34


,user_id,date,sbp,dbp
0,100000,2017-04-24,153,108
1,100000,2017-04-25,152,106
2,100000,2017-04-26,151,106
3,100000,2017-04-27,154,109
4,100000,2017-04-28,155,100


In [7]:
bp_grouped = blood_pressure_df[['user_id', 'date']].groupby('user_id')
start_dates = bp_grouped.min().rename(columns={'date':'start_date'})
end_dates = bp_grouped.max().rename(columns={'date':'end_date'})
dates = start_dates.join(end_dates)
dates['Total_days_on_proteus'] = dates['end_date'] - dates['start_date']
user_df1 = user_df.merge(dates.reset_index(), on='user_id')
display(user_df1.head())
display(user_df1.describe())

,user_id,gender,bmi,start_date,end_date,Total_days_on_proteus
0,100000,female,39,2017-04-24,2017-06-22,59 days
1,100001,male,25,2017-04-17,2017-06-15,59 days
2,100002,female,28,2017-05-06,2017-07-04,59 days
3,100003,male,25,2017-04-25,2017-06-22,58 days
4,100004,female,34,2017-04-24,2017-06-22,59 days


,user_id,bmi,Total_days_on_proteus
count,4410.000000,4410.000000,4410
mean,102204.500000,28.691837,58 days 22:10:36.734693
std,1273.201673,7.200908,0 days 06:41:57.930042
min,100000.000000,18.000000,56 days 00:00:00
25%,101102.250000,25.000000,59 days 00:00:00
50%,102204.500000,27.000000,59 days 00:00:00
75%,103306.750000,31.000000,59 days 00:00:00
max,104409.000000,100.000000,59 days 00:00:00


In [12]:
under_weight_users_df = user_df1[user_df1.bmi <= 18.5]
healthy_weight_df = user_df1[(user_df1.bmi > 18.5) & (user_df1.bmi <= 25)]
over_weight_df = user_df1[(user_df1.bmi > 25) & (user_df1.bmi <= 30)]
obese_class1_df = user_df1[(user_df1.bmi > 30) & (user_df1.bmi <= 35)]
obese_class2_df = user_df1[(user_df1.bmi > 35) & (user_df1.bmi <= 40)]
obese_class3_df = user_df1[user_df1.bmi > 40]
print(len(under_weight_users_df), len(healthy_weight_df), len(over_weight_df), len(obese_class1_df), 
     len(obese_class2_df), len(obese_class3_df))


3 1414 1786 785 280 142


In [6]:
print(len(blood_pressure_df), len(medication_df), len(user_df))

255162 255107 4410


In [15]:
bp_medication_df = blood_pressure_df.merge(medication_df, on=['user_id', 'date'], how='outer')
drop_users = bp_medication_df[(bp_medication_df['pill_detect'].isna()) | (bp_medication_df['sbp'].isna()) | 
                 (bp_medication_df['dbp'].isna())]['user_id'].unique()
print(drop_users)
display(bp_medication_df[(bp_medication_df['sbp'].isna()) | (bp_medication_df['dbp'].isna())])

[102465 102951 103550 104225 100005 102308 102342 103759 103761 104086]


,user_id,date,sbp,dbp,pill_detect
255162,100005,5/3/17,NaN,NaN,False
255163,102308,5/5/17,NaN,NaN,False
255164,102342,6/14/17,NaN,NaN,True
255165,103759,5/22/17,NaN,NaN,False
255166,103761,5/28/17,NaN,NaN,True
255167,104086,5/20/17,NaN,NaN,True


In [13]:
bp_medication_df[drop]

,user_id,date,sbp,dbp,pill_detect
255162,100005,5/3/17,NaN,NaN,False
255163,102308,5/5/17,NaN,NaN,False
255164,102342,6/14/17,NaN,NaN,True
255165,103759,5/22/17,NaN,NaN,False
255166,103761,5/28/17,NaN,NaN,True


In [62]:
dates.dtypes

start_date                datetime64[ns]
end_date                  datetime64[ns]
Total_days_on_proteus    timedelta64[ns]
dtype: object